In [1]:
# # Install required libraries
# import os

# try:
#     import selenium
# except ImportError:
#     os.system("pip install selenium")

# try:
#     import webdriver_manager
# except ImportError:
#     os.system("pip install webdriver-manager")

# try:
#     import pandas as pd
# except ImportError:
#     os.system("pip install pandas")

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

In [4]:
# Initialize Chrome with options
chrome_options = Options()
chrome_options.add_argument("--start-maximized")  # Start maximized for better visibility
chrome_options.add_argument("--disable-infobars")  # Disable infobars
chrome_options.add_argument("--disable-extensions")  # Disable extensions for simplicity
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

def scroll_to_bottom(driver, pause_time=2):
    """
    Scroll to the bottom of the page to load dynamic content.
    """
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(pause_time)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

driver.get("https://www.expedia.com/Hotel-Search?adults=2&children=&d1=2025-05-01&d2=2025-05-07&destination=Auckland%20%28and%20vicinity%29%2C%20Auckland%20Region%2C%20New%20Zealand&endDate=2025-05-07&latLong=-36.84906%2C174.76538&mapBounds=&pwaDialog=&regionId=178233&rooms=1&semdtl=&sort=PROPERTY_CLASS&startDate=2025-05-01&stay_options_group=hotels_option&theme=&useRewards=false&userIntent=")
time.sleep(6)  # Wait for the page to load completely

# Extract hotel data
listing_container = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-stid="property-listing-results"]'))
)
hotel_links = listing_container.find_elements(By.CSS_SELECTOR, 'a[data-stid="open-hotel-information"]')
print(f"Found {len(hotel_links)} hotel links.")


Found 253 hotel links.


In [5]:
import pandas as pd

# Initialize a list to store all hotel data
hotel_data = []
location = "Auckland, New Zealand"

for index, link in enumerate(hotel_links):
    try:
        href = link.get_attribute('href')
        print(f"\nProcessing link {index + 1}")

        # Open the link in a new tab using JavaScript
        driver.execute_script("window.open(arguments[0]);", href)
        time.sleep(3)  # Wait for the new tab to open

        # Switch to the new tab
        driver.switch_to.window(driver.window_handles[-1])
        time.sleep(15)  # Wait for the new page to load

        hotel_details = {"Location": location}

        # Extract Hotel Name
        try:
            # First, try to locate the hotel name with the old structure
            try:
                hotel_name_element = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "h1.uitk-heading.uitk-heading-3"))
                )
                hotel_details['Hotel Name'] = hotel_name_element.text.strip()
            except Exception:
                # If the old structure fails, try the new structure
                hotel_name_element = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "h1.uitk-heading.uitk-heading-4"))
                )
                hotel_details['Hotel Name'] = hotel_name_element.text.strip()

            print(f"Extracted Hotel Name: {hotel_details['Hotel Name']}")
        except Exception as e:
            print(f"Error extracting hotel name: {e}")
            hotel_details['Hotel Name'] = None

        # Extract Stars
        try:
            stars_container = driver.find_element(By.CSS_SELECTOR, "div.uitk-rating.uitk-spacing.uitk-spacing-margin-blockstart-one")
            stars_count = len(stars_container.find_elements(By.TAG_NAME, "svg"))
            hotel_details['Stars'] = stars_count
        except Exception as e:
            print(f"Error extracting stars for hotel {index + 1}: {e}")
            hotel_details['Stars'] = None

        # Extract Rating
        try:
            rating_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "span.uitk-badge-base-large.uitk-badge-positive > span.uitk-badge-base-text")
                )
            )
            hotel_details['Rating'] = rating_element.text.strip()
        except Exception as e:
            print(f"Error extracting rating: {e}")
            hotel_details['Rating'] = None

        # Extract Review Keyword
        try:
            review_summary_container = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-stid='content-hotel-reviewsummary']"))
            )
            review_keyword_element = review_summary_container.find_element(
                By.CSS_SELECTOR, "div.uitk-text.uitk-type-500.uitk-type-medium.uitk-text-default-theme"
            )
            review_keyword = review_keyword_element.text.strip()
            hotel_details['Review Keyword'] = review_keyword
            print(f"Extracted Review Keyword: {review_keyword}")
        except Exception as e:
            print(f"Error extracting review keyword: {e}")
            hotel_details['Review Keyword'] = None

        # Extract Services
        try:
            services_list = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "ul.uitk-typelist[data-stid='sp-content-list']"))
            )

            services_lis = services_list.find_elements(By.CSS_SELECTOR, "li[data-stid^='sp-content-item-']")
            service_count = len(services_lis)
            print(f"Number of services: {service_count}")

            services = []
            for i in range(service_count):
                try:
                    service_span = driver.find_element(
                        By.CSS_SELECTOR,
                        f"ul.uitk-typelist[data-stid='sp-content-list'] li[data-stid='sp-content-item-{i}'] span.uitk-text.uitk-type-300.uitk-text-default-theme.uitk-layout-flex-item"
                    )
                    services.append(service_span.text.strip())
                except Exception as e:
                    print(f"Error extracting service {i}: {e}")
                    services.append(None)

            hotel_details['Services'] = services
        except Exception as e:
            print(f"Error extracting services: {e}")
            hotel_details['Services'] = []

        # Extract Property Offers
        try:
            property_offers_container = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-stid="section-room-list"]'))
            )

            property_offers = property_offers_container.find_elements(By.CSS_SELECTOR, 'div[data-stid^="property-offer-"]')

            offers = []
            for offer_index, offer in enumerate(property_offers, start=1):
                offer_details = {}

                try:
                    room_name_element = offer.find_element(By.CSS_SELECTOR, "h3.uitk-heading.uitk-heading-6")
                    offer_details['Room Name'] = room_name_element.text.strip()
                except Exception as e:
                    print(f"Error extracting room name for offer {offer_index}: {e}")
                    offer_details['Room Name'] = None

                try:
                    price_summary_div = offer.find_element(By.CSS_SELECTOR, 'div[data-stid="price-summary"]')
                    room_charges_element = price_summary_div.find_element(
                        By.CSS_SELECTOR,
                        "div.uitk-text.uitk-type-500.uitk-type-medium.uitk-text-emphasis-theme"
                    )
                    offer_details['Room Charges'] = room_charges_element.text.strip()
                except Exception as e:
                    print(f"Error extracting room charges for offer {offer_index}: {e}")
                    offer_details['Room Charges'] = None

                offers.append(offer_details)

            hotel_details['Property Offers'] = offers
        except Exception as e:
            print(f"Error extracting property offers: {e}")
            hotel_details['Property Offers'] = []

        # Add hotel details to list
        hotel_data.append(hotel_details)
        print(f"Extracted Hotel Data: {hotel_details}")

        # Close the new tab
        driver.close()

        # Switch back to the main tab
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(1)

    except Exception as e:
        print(f"Error processing link {index + 1}: {e}")
        continue

# Export hotel data to a CSV file
df = pd.DataFrame(hotel_data)

# Normalize property offers if necessary
if 'Property Offers' in df.columns:
    df['Property Offers'] = df['Property Offers'].apply(lambda x: ', '.join(str(offer) for offer in x) if isinstance(x, list) else x)

# Save to CSV
output_filename = f"hotel_data_{location.replace(',', '').replace(' ', '_')}.csv"
df.to_csv(output_filename, index=False)
print(f"Data exported to {output_filename}")
# Export hotel data to a CSV file
df = pd.DataFrame(hotel_data)


Processing link 1
Error extracting hotel name: Message: 
Stacktrace:
	GetHandleVerifier [0x007134A3+25059]
	(No symbol) [0x0069CEA4]
	(No symbol) [0x0057BEC3]
	(No symbol) [0x005BFD86]
	(No symbol) [0x005BFFCB]
	(No symbol) [0x005FD952]
	(No symbol) [0x005E1F44]
	(No symbol) [0x005FB51E]
	(No symbol) [0x005E1C96]
	(No symbol) [0x005B3FAC]
	(No symbol) [0x005B4F3D]
	GetHandleVerifier [0x00A05613+3113811]
	GetHandleVerifier [0x00A1A2DA+3199002]
	GetHandleVerifier [0x00A12AB2+3168242]
	GetHandleVerifier [0x007B3310+680016]
	(No symbol) [0x006A57ED]
	(No symbol) [0x006A2A98]
	(No symbol) [0x006A2C35]
	(No symbol) [0x00695890]
	BaseThreadInitThunk [0x76795D49+25]
	RtlInitializeExceptionChain [0x77C1CEBB+107]
	RtlGetAppContainerNamedObjectPath [0x77C1CE41+561]

Error extracting stars for hotel 1: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.uitk-rating.uitk-spacing.uitk-spacing-margin-blockstart-one"}
  (Session info: chrome=131.0.6778.109); F


Processing link 66
Extracted Review Keyword: Wonderful
Number of services: 6
Extracted Hotel Data: {'Location': 'Washington, USA', 'Hotel Name': 'The Darcy Hotel', 'Stars': 5, 'Rating': '9.2', 'Review Keyword': 'Wonderful', 'Services': ['Restaurant', 'Breakfast available', 'Bar', 'Room service', 'Housekeeping', 'Laundry'], 'Property Offers': [{'Room Name': 'Room, 2 Queen Beds', 'Room Charges': '$146'}, {'Room Name': 'City View, 1 King Bed', 'Room Charges': '$156'}, {'Room Name': 'Room, 2 Queen Beds, Accessible (Hearing)', 'Room Charges': '$146'}, {'Room Name': 'City View, 1 King Bed, Accessible (Hearing)', 'Room Charges': '$156'}, {'Room Name': 'City View, 2 Queen Beds, Accessible (Roll-In Shower)', 'Room Charges': '$162'}, {'Room Name': 'City View, 2 Queen Beds, Accessible (Bathtub)', 'Room Charges': '$162'}, {'Room Name': 'Room, 2 Queen Beds, City View', 'Room Charges': '$162'}, {'Room Name': 'Studio Suite, City View', 'Room Charges': '$220'}, {'Room Name': 'Suite, 1 Bedroom (Roll-i


Processing link 68
Extracted Review Keyword: Very good
Number of services: 6
Error extracting room charges for offer 22: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[data-stid="price-summary"]"}
  (Session info: chrome=131.0.6778.140); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00B334A3+25059]
	(No symbol) [0x00ABCEA4]
	(No symbol) [0x0099BEC3]
	(No symbol) [0x009DFD86]
	(No symbol) [0x009DFFCB]
	(No symbol) [0x009D5B01]
	(No symbol) [0x00A01F44]
	(No symbol) [0x009D5A24]
	(No symbol) [0x00A02194]
	(No symbol) [0x00A1B51E]
	(No symbol) [0x00A01C96]
	(No symbol) [0x009D3FAC]
	(No symbol) [0x009D4F3D]
	GetHandleVerifier [0x00E25613+3113811]
	GetHandleVerifier [0x00E3A2DA+3199002]
	GetHandleVerifier [0x00E32AB2+3168242]
	GetHandleVerifier [0x00BD3310+680016]
	(No symbol) [0x00AC57ED]
	(No symbol) [0x00AC2A9


Processing link 69
Extracted Review Keyword: Excellent
Number of services: 6
Error extracting room charges for offer 6: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[data-stid="price-summary"]"}
  (Session info: chrome=131.0.6778.140); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00B334A3+25059]
	(No symbol) [0x00ABCEA4]
	(No symbol) [0x0099BEC3]
	(No symbol) [0x009DFD86]
	(No symbol) [0x009DFFCB]
	(No symbol) [0x009D5B01]
	(No symbol) [0x00A01F44]
	(No symbol) [0x009D5A24]
	(No symbol) [0x00A02194]
	(No symbol) [0x00A1B51E]
	(No symbol) [0x00A01C96]
	(No symbol) [0x009D3FAC]
	(No symbol) [0x009D4F3D]
	GetHandleVerifier [0x00E25613+3113811]
	GetHandleVerifier [0x00E3A2DA+3199002]
	GetHandleVerifier [0x00E32AB2+3168242]
	GetHandleVerifier [0x00BD3310+680016]
	(No symbol) [0x00AC57ED]
	(No symbol) [0x00AC2A98


Processing link 70
Extracted Review Keyword: Excellent
Number of services: 6
Extracted Hotel Data: {'Location': 'Washington, USA', 'Hotel Name': 'Hyatt House Washington DC/The Wharf', 'Stars': 4, 'Rating': '8.8', 'Review Keyword': 'Excellent', 'Services': ['Breakfast included', 'Restaurant', 'Pool', 'Gym', 'Housekeeping', 'Bar'], 'Property Offers': [{'Room Name': 'Studio, 1 King Bed with Sofa bed, River View', 'Room Charges': '$237'}, {'Room Name': 'Room, 1 Bedroom, Kitchen (1 King Bed and 1 Sofa Bed)', 'Room Charges': '$315'}, {'Room Name': 'Room, 1 Bedroom, Accessible, Bathtub (1 King Bed and 1 Sofa Bed)', 'Room Charges': '$315'}, {'Room Name': 'Room, 1 Bedroom (1 King Bed and 1 Sofa Bed)', 'Room Charges': '$448'}]}

Processing link 71
Extracted Review Keyword: Wonderful
Number of services: 6
Extracted Hotel Data: {'Location': 'Washington, USA', 'Hotel Name': 'Viceroy Washington DC', 'Stars': 5, 'Rating': '9.2', 'Review Keyword': 'Wonderful', 'Services': ['Restaurant', 'Pool', 'Bar'

...


Processing link 94
Extracted Review Keyword: Very good
Number of services: 6
Extracted Hotel Data: {'Location': 'Washington, USA', 'Hotel Name': 'Hyatt Regency Washington on Capitol Hill', 'Stars': 4, 'Rating': '8.4', 'Review Keyword': 'Very good', 'Services': ['Housekeeping', 'Breakfast available', 'Bar', 'Pool', 'Laundry', 'Gym'], 'Property Offers': [{'Room Name': 'Room, 1 King Bed', 'Room Charges': '$204'}, {'Room Name': 'Room, 2 Double Beds', 'Room Charges': '$204'}, {'Room Name': 'Room, 2 Double Beds, Accessible (Shower)', 'Room Charges': '$204'}, {'Room Name': 'Room, 1 King Bed, Accessible, Bathtub', 'Room Charges': '$204'}, {'Room Name': 'Room, 1 King Bed (High Floor)', 'Room Charges': '$237'}, {'Room Name': 'Room, 1 King Bed, Corner', 'Room Charges': '$337'}, {'Room Name': 'Executive Room', 'Room Charges': '$337'}, {'Room Name': 'Suite (CAPITOL)', 'Room Charges': '$467'}]}

Processing link 95
Extracted Review Keyword: Very good
Number of services: 6
Error extracting room charg


Processing link 96
Extracted Review Keyword: Very good
Number of services: 6
Extracted Hotel Data: {'Location': 'Washington, USA', 'Hotel Name': 'Econo Lodge Metro', 'Stars': 2, 'Rating': '8.0', 'Review Keyword': 'Very good', 'Services': ['Breakfast included', 'Air conditioning', '24/7 front desk', 'Bar', 'Laundry', 'Housekeeping'], 'Property Offers': [{'Room Name': 'Standard Room, 1 King Bed, Non Smoking', 'Room Charges': '$99'}, {'Room Name': 'Standard Room, 2 Double Beds, Non Smoking', 'Room Charges': '$99'}, {'Room Name': 'Room, 1 Queen Bed, Non Smoking', 'Room Charges': '$99'}, {'Room Name': 'Room, 1 King Bed with Sofa bed, Non Smoking', 'Room Charges': '$129'}]}

Processing link 97
Extracted Review Keyword: Wonderful
Number of services: 6
Error extracting room charges for offer 5: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[data-stid="price-summary"]"}
  (Session info: chrome=131.0.6778.140); For documentation on this error, pleas


Processing link 98
Extracted Review Keyword: Very good
Number of services: 6
Error extracting room charges for offer 7: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[data-stid="price-summary"]"}
  (Session info: chrome=131.0.6778.140); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00B334A3+25059]
	(No symbol) [0x00ABCEA4]
	(No symbol) [0x0099BEC3]
	(No symbol) [0x009DFD86]
	(No symbol) [0x009DFFCB]
	(No symbol) [0x009D5B01]
	(No symbol) [0x00A01F44]
	(No symbol) [0x009D5A24]
	(No symbol) [0x00A02194]
	(No symbol) [0x00A1B51E]
	(No symbol) [0x00A01C96]
	(No symbol) [0x009D3FAC]
	(No symbol) [0x009D4F3D]
	GetHandleVerifier [0x00E25613+3113811]
	GetHandleVerifier [0x00E3A2DA+3199002]
	GetHandleVerifier [0x00E32AB2+3168242]
	GetHandleVerifier [0x00BD3310+680016]
	(No symbol) [0x00AC57ED]
	(No symbol) [0x00AC2A98


Processing link 99
Extracted Review Keyword: Excellent
Number of services: 6
Error extracting service 1: Message: no such element: Unable to locate element: {"method":"css selector","selector":"ul.uitk-typelist[data-stid='sp-content-list'] li[data-stid='sp-content-item-1'] span.uitk-text.uitk-type-300.uitk-text-default-theme.uitk-layout-flex-item"}
  (Session info: chrome=131.0.6778.140); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00B334A3+25059]
	(No symbol) [0x00ABCEA4]
	(No symbol) [0x0099BEC3]
	(No symbol) [0x009DFD86]
	(No symbol) [0x009DFFCB]
	(No symbol) [0x00A1D952]
	(No symbol) [0x00A01F44]
	(No symbol) [0x00A1B51E]
	(No symbol) [0x00A01C96]
	(No symbol) [0x009D3FAC]
	(No symbol) [0x009D4F3D]
	GetHandleVerifier [0x00E25613+3113811]
	GetHandleVerifier [0x00E3A2DA+3199002]
	GetHandleVerifier [0x00E32AB2+3168242]
	GetHandleVerifier [0x00BD3310+6


Processing link 100
Extracted Review Keyword: Wonderful
Number of services: 6
Extracted Hotel Data: {'Location': 'Washington, USA', 'Hotel Name': 'Thompson Washington D.C., by Hyatt', 'Stars': 4, 'Rating': '9.2', 'Review Keyword': 'Wonderful', 'Services': ['Restaurant', 'Bar', 'Housekeeping', 'Breakfast available', 'Pet friendly', 'Laundry'], 'Property Offers': [{'Room Name': 'Room, 1 King Bed', 'Room Charges': '$159'}, {'Room Name': 'Room, 1 Queen Bed', 'Room Charges': '$159'}, {'Room Name': 'Room, 1 King Bed, Balcony', 'Room Charges': '$174'}, {'Room Name': 'Room, 1 King Bed, Corner', 'Room Charges': '$184'}]}
Data exported to hotel_data_Washington_USA.csv
